In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow as tf
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import load_model
from sklearn.model_selection import StratifiedKFold
import os
from keras.callbacks import ModelCheckpoint, EarlyStopping

%matplotlib inline

Using TensorFlow backend.


In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/Azure-Samples/Machine-Learning-Operationalization/master/samples/spark/tutorials/datasets/housing.csv",
                )

In [3]:
df.head(5)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,5.33,36.2


- CRIM: 인구 1인당 범죄 발생 수
- ZN: 25,000평방 피트 이상의 주거 구역 비중
- INDUS: 소매업 외 상업이 차지하는 면적 비율
- CHAS: 찰스강 위치 변수(1: 강 주변, 0: 이외)
- NOX: 일산화질소 농도
- RM: 집의 평균 방 수
- AGE: 1940년 이전에 지어진 비율
- DIS: 5가지 보스턴 시 고용 시설까지의 거리
- RAD: 순환고속도로의 접근 용이성
- TAX: `$10,000`당 부동산 세율 총계
- PTRATIO: 지역별 학생과 교사 비율
- ~~B: 지역별 흑인 비율~~
- LSTAT: 급여가 낮은 직업에 종사하는 인구 비율(%)
- 가격(단위: `$1,000`)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     506 non-null    float64
 1   ZN       506 non-null    float64
 2   INDUS    506 non-null    float64
 3   CHAS     506 non-null    int64  
 4   NOX      506 non-null    float64
 5   RM       506 non-null    float64
 6   AGE      506 non-null    float64
 7   DIS      506 non-null    float64
 8   RAD      506 non-null    int64  
 9   TAX      506 non-null    int64  
 10  PTRATIO  506 non-null    float64
 11  LSTAT    506 non-null    float64
 12  MEDV     506 non-null    float64
dtypes: float64(10), int64(3)
memory usage: 51.5 KB


In [5]:
seed = 0
np.random.seed(seed)
tf.random.set_seed(3)

In [6]:
dataset = df.values
X = dataset[:, :12]
Y = dataset[:, 12]

In [7]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=seed)

In [8]:
model = Sequential()
model.add(Dense(30, input_dim=12, activation='relu'))
model.add(Dense(6, activation='relu'))
model.add(Dense(1))

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 30)                390       
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 186       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 7         
Total params: 583
Trainable params: 583
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X_train, Y_train, epochs=200, batch_size=10)

Train on 354 samples
Epoch 1/200
354/354 [==============================] - 1s 1ms/sample - loss: 2305.0962
Epoch 2/200
354/354 [==============================] - 0s 178us/sample - loss: 766.3044
Epoch 3/200
354/354 [==============================] - 0s 162us/sample - loss: 646.8615
Epoch 4/200
354/354 [==============================] - 0s 165us/sample - loss: 609.1120
Epoch 5/200
354/354 [==============================] - 0s 167us/sample - loss: 597.6417
Epoch 6/200
354/354 [==============================] - 0s 159us/sample - loss: 595.9057
Epoch 7/200
354/354 [==============================] - 0s 165us/sample - loss: 594.4369
Epoch 8/200
354/354 [==============================] - 0s 154us/sample - loss: 592.9878
Epoch 9/200
354/354 [==============================] - 0s 146us/sample - loss: 591.5506
Epoch 10/200
354/354 [==============================] - 0s 157us/sample - loss: 590.1423
Epoch 11/200
354/354 [==============================] - 0s 162us/sample - loss: 588.7368
Epoch 12/2

Epoch 93/200
354/354 [==============================] - 0s 139us/sample - loss: 471.6702
Epoch 94/200
354/354 [==============================] - 0s 270us/sample - loss: 470.3271
Epoch 95/200
354/354 [==============================] - 0s 261us/sample - loss: 468.9972
Epoch 96/200
354/354 [==============================] - 0s 139us/sample - loss: 467.6663
Epoch 97/200
354/354 [==============================] - 0s 139us/sample - loss: 466.3347
Epoch 98/200
354/354 [==============================] - 0s 137us/sample - loss: 465.0045
Epoch 99/200
354/354 [==============================] - 0s 138us/sample - loss: 463.6795
Epoch 100/200
354/354 [==============================] - 0s 133us/sample - loss: 462.3562
Epoch 101/200
354/354 [==============================] - 0s 139us/sample - loss: 461.0433
Epoch 102/200
354/354 [==============================] - 0s 135us/sample - loss: 459.7231
Epoch 103/200
354/354 [==============================] - 0s 141us/sample - loss: 458.4098
Epoch 104/200
354

354/354 [==============================] - 0s 138us/sample - loss: 360.0263
Epoch 185/200
354/354 [==============================] - 0s 144us/sample - loss: 358.9042
Epoch 186/200
354/354 [==============================] - 0s 134us/sample - loss: 357.7928
Epoch 187/200
354/354 [==============================] - 0s 136us/sample - loss: 356.6778
Epoch 188/200
354/354 [==============================] - 0s 133us/sample - loss: 355.5701
Epoch 189/200
354/354 [==============================] - 0s 136us/sample - loss: 354.4578
Epoch 190/200
354/354 [==============================] - 0s 141us/sample - loss: 353.3570
Epoch 191/200
354/354 [==============================] - 0s 139us/sample - loss: 352.2628
Epoch 192/200
354/354 [==============================] - 0s 131us/sample - loss: 351.1668
Epoch 193/200
354/354 [==============================] - 0s 129us/sample - loss: 350.0810
Epoch 194/200
354/354 [==============================] - 0s 134us/sample - loss: 348.9757
Epoch 195/200
354/354 [=

In [11]:
Y_pred = model.predict(X_test).flatten()

In [12]:
for i in range(10):
    real = Y_test[i]
    pred = Y_pred[i]
    print("Real price: {:.3f}, pred price: {:.3f}".format(real, pred))

Real price: 22.600, pred price: 6.710
Real price: 50.000, pred price: 6.710
Real price: 23.000, pred price: 6.710
Real price: 8.300, pred price: 6.710
Real price: 21.200, pred price: 6.710
Real price: 19.900, pred price: 6.710
Real price: 20.600, pred price: 6.710
Real price: 18.700, pred price: 6.710
Real price: 16.100, pred price: 6.710
Real price: 18.600, pred price: 6.710
